In [40]:
import pandas as pd
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

nfl = pd.read_csv("NFL_data_super_cleaned.csv")

In [41]:
# convert to an array
play = array(nfl["play_type"])

# encode as integers
play_encoder = LabelEncoder()
play_encoded =  play_encoder.fit_transform(play) 
 
# binary encode
play_onehot_encoder = OneHotEncoder(sparse=False) # disable sparse return type
# reshape the array
play_encoded = play_encoded.reshape(len(play_encoded), 1) 
play_onehot_encoded = play_onehot_encoder.fit_transform(play_encoded)

nfl["play_type"] = play_onehot_encoded.tolist()

posteam = array(nfl["posteam"])

# encode as integers
posteam_encoder = LabelEncoder()
posteam_encoded =  posteam_encoder.fit_transform(posteam) 
 
# binary encode
posteam_onehot_encoder = OneHotEncoder(sparse=False) # disable sparse return type
# reshape the array
posteam_encoded = posteam_encoded.reshape(len(posteam_encoded), 1) 
posteam_onehot_encoded = pd.DataFrame(posteam_onehot_encoder.fit_transform(posteam_encoded))

nfl

,posteam,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_margin
0,PIT,58.0,893.0,1,1.0,0.0,10,"[0.0, 1.0, 0.0]",0.0
1,PIT,53.0,856.0,1,2.0,0.0,5,"[0.0, 0.0, 1.0]",0.0
2,PIT,56.0,815.0,1,3.0,0.0,8,"[0.0, 1.0, 0.0]",0.0
3,PIT,56.0,807.0,1,4.0,0.0,8,"[1.0, 0.0, 0.0]",0.0
4,TEN,98.0,796.0,1,1.0,0.0,10,"[0.0, 0.0, 1.0]",0.0
...,...,...,...,...,...,...,...,...,...
353055,CAR,71.0,82.0,4,2.0,0.0,1,"[0.0, 1.0, 0.0]",-5.0
353056,CAR,71.0,77.0,4,3.0,0.0,1,"[0.0, 1.0, 0.0]",-5.0
353057,CAR,66.0,63.0,4,2.0,0.0,10,"[0.0, 1.0, 0.0]",-5.0
353058,CAR,66.0,58.0,4,3.0,0.0,10,"[0.0, 1.0, 0.0]",-5.0


In [45]:
from sklearn.preprocessing import MinMaxScaler
nfl1 = nfl.drop(columns = ['posteam'])
nfl2 = posteam_onehot_encoded.join(nfl1)
nfl2.columns = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 
'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 
'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS', 'yardline_100', 'quarter_seconds_remaining', 
'qtr', 'down', 'goal_to_go', 'ydstogo', 'play_type', 'score_margin']
# Min-max scaler
X = nfl2.drop("play_type", axis = 1)
y = play_onehot_encoded
X = (X-X.min())/(X.max()-X.min())
'''
scaler = MinMaxScaler(feature_range=[0, 1])
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)
#print(X)


set_of_classes = y.value_counts().index.tolist()
set_of_classes= pd.DataFrame({'Class': set_of_classes})
y = pd.get_dummies(y)
y'''

"\nscaler = MinMaxScaler(feature_range=[0, 1])\nX_rescaled = scaler.fit_transform(X)\nX = pd.DataFrame(data = X_rescaled, columns = X.columns)\n#print(X)\n\n\nset_of_classes = y.value_counts().index.tolist()\nset_of_classes= pd.DataFrame({'Class': set_of_classes})\ny = pd.get_dummies(y)\ny"

In [46]:
print(X.columns[X.isna().any()])
X["down"] = X["down"].fillna(0)

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 100, hidden_layer_sizes = (12, 3), max_iter = 500)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test) 

Index(['down'], dtype='object')


In [47]:
y_pred

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [48]:
#Generate a confusion matrix
#print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

print("Accuracy:", clf.score(X_test, y_test))

mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

print(classification_report(y_test, y_pred))

Accuracy: 0.6831133518382144
MSE: 0.20253403576351517
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      3354
           1       0.66      0.88      0.76     18660
           2       0.69      0.40      0.50     13292

   micro avg       0.69      0.71      0.70     35306
   macro avg       0.75      0.75      0.73     35306
weighted avg       0.70      0.71      0.68     35306
 samples avg       0.69      0.71      0.70     35306



c:\Users\hansu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
